In [21]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [22]:
df=pd.read_csv('hotel_bookings.csv')
data1 = df.copy()
data1 = data1.drop(['company'], axis = 1)
data1 = data1.dropna(axis = 0)
target = data1[['adr']]
data1 = data1.drop('adr',axis = 1)
col = list(data1.columns)
def get_cols_with_no_nans(df,col_type):
    if (col_type == 'num'):
        predictors = df.select_dtypes(exclude=['object'])
    elif (col_type == 'no_num'):
        predictors = df.select_dtypes(include=['object'])
    elif (col_type == 'all'):
        predictors = df
    else :
        print('Error : choose a type (num, no_num, all)')
        return 0
    cols_with_no_nans = []
    for col in predictors.columns:
        if not df[col].isnull().any():
            cols_with_no_nans.append(col)
    return cols_with_no_nans
cat_cols = get_cols_with_no_nans(data1 , 'no_num')
def oneHotEncode(df,colNames):
    for col in colNames:
        if( df[col].dtype == np.dtype('object')):
            dummies = pd.get_dummies(df[col],prefix=col)
            df = pd.concat([df,dummies],axis=1)

            #drop the encoded column
            df.drop([col],axis = 1 , inplace=True)
    return df
    
print('There were {} columns before encoding categorical features'.format(data1.shape[1]))
new_data = oneHotEncode(data1, cat_cols)
print('There are {} columns after encoding categorical features'.format(new_data.shape[1]))
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(new_data, target, test_size=0.33, random_state=42)

In [73]:
model=keras.Sequential(
    [
        layers.Dense(30,activation='relu',kernel_initializer='normal',input_dim = X_train.shape[1]),
        layers.Dense(512,activation='relu',kernel_initializer='normal'),
        layers.Dense(512,activation='relu',kernel_initializer='normal'),
        layers.Dense(512,activation='relu',kernel_initializer='normal'),
        layers.Dense(1,activation='linear',kernel_initializer='normal')
    ]
)

In [74]:
model.compile(
    optimizer='adam',
    loss='mean_absolute_error',
    metrics=['mean_absolute_error']
)

In [75]:
model.fit(X_train, y_train, epochs=100, batch_size=32)
model.evaluate(X_test,y_test)

Train on 68938 samples
Epoch 1/100
68938/68938 [==============================] - 13s 182us/sample - loss: 28.1962 - mean_absolute_error: 28.1962
Epoch 2/100
68938/68938 [==============================] - 12s 179us/sample - loss: 20.4433 - mean_absolute_error: 20.4434
Epoch 3/100
68938/68938 [==============================] - 15s 219us/sample - loss: 18.2802 - mean_absolute_error: 18.2802
Epoch 4/100
68938/68938 [==============================] - 12s 174us/sample - loss: 17.3237 - mean_absolute_error: 17.3238
Epoch 5/100
68938/68938 [==============================] - 12s 173us/sample - loss: 16.6283 - mean_absolute_error: 16.6283
Epoch 6/100
68938/68938 [==============================] - 12s 173us/sample - loss: 16.2659 - mean_absolute_error: 16.2659
Epoch 7/100
68938/68938 [==============================] - 12s 172us/sample - loss: 15.7807 - mean_absolute_error: 15.7807
Epoch 8/100
68938/68938 [==============================] - 15s 213us/sample - loss: 15.4965 - mean_absolute_error: 1

Epoch 67/100
68938/68938 [==============================] - 14s 210us/sample - loss: 10.7736 - mean_absolute_error: 10.7736
Epoch 68/100
68938/68938 [==============================] - 14s 210us/sample - loss: 10.7561 - mean_absolute_error: 10.7561
Epoch 69/100
68938/68938 [==============================] - 14s 209us/sample - loss: 10.6841 - mean_absolute_error: 10.6841
Epoch 70/100
68938/68938 [==============================] - 14s 209us/sample - loss: 10.6520 - mean_absolute_error: 10.6520
Epoch 71/100
68938/68938 [==============================] - 15s 211us/sample - loss: 10.6605 - mean_absolute_error: 10.6605
Epoch 72/100
68938/68938 [==============================] - 15s 211us/sample - loss: 10.6638 - mean_absolute_error: 10.6638
Epoch 73/100
68938/68938 [==============================] - 14s 210us/sample - loss: 10.6409 - mean_absolute_error: 10.6408
Epoch 74/100
68938/68938 [==============================] - 15s 211us/sample - loss: 10.6742 - mean_absolute_error: 10.6742
Epoch 75

[11.327010085633567, 11.327017]

In [76]:
y_pred=model.predict(X_test)
r2_score(y_test,y_pred)

0.8466206758368364